# Stacking 연습 - breast_cancer 데이터

##### CV 세트 기반 스태킹 알고리즘까지 스태킹을 이해하기 위해 하는 스태킹 실습.  해당 스태킹 모델을 활용하여 유방암 여부를 예측하는 모형을 만들어본다.

##### 해당 링크를 참조하여 만들어본다. https://kimdingko-world.tistory.com/186 

---

CV Set 기반 스태킹 전 일반 스태킹을 진행한 다음 비교해보고자 한다.

첫 번째로는 블로그 내용에서는 따로 표준화를 진행하지 않았으므로 그대로 따라서 결과 데이터를 행렬 상태로 이용하여 진행하였고,

두 번째로는 표준화 진행하지 않은 채로 StackingClassifier를 이용하기로 한다.

세 번째로는 표준화를 진행하고 행렬 상태를 이용하도록 하고, 

네 번쨰로는 표준화를 진행하고 StackingClassifier를 이용하도록 하겠다.

그다음 CV set 기반 스태킹을 실습하려 한다. 


> 배운점: 사이킷런의 스태킹모델로 스태킹을 할 때와, 결과들을 수제로 데이터 넣는 것과는 차이가 있다. 


## Dataset import

### 데이터 불러오기

In [1]:
from sklearn import datasets
raw_breast_cancer = datasets.load_breast_cancer()

In [2]:
# 데이터 셋 내 피처 살펴보기
raw_breast_cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

### 피처, 타깃 데이터 지정

In [3]:
X_data = raw_breast_cancer.data
y_label = raw_breast_cancer.target

# Mode1: No Standardization

### 트레이닝/테스트 데이터 분할

In [4]:
from sklearn.model_selection import train_test_split                # 데이터 분할을 위한 함수
# from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X_data, y_label, random_state = 0) # 데이터 분할

# std_scale = StandardScaler()                                      # 표준화 스케일러 선언
# std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

# X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
# X_te_std = std_scale.transform(X_te)

## Model1: 베이스러너 아웃풋을 전치시켜 메타러너에 입력

### 데이터 학습

In [13]:
# 스태킹 모델에 사용할 알고리즘 호출
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# 베이스 러너: 개별 ML 모델 객체 생성 
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier(random_state = 0)
ada_clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)

# 메타 모델: 스태킹으로 만들어진 데이터 학습 및 예측 - 로지스틱 리그레션 모형
lr_final = LogisticRegression(C = 10, random_state = 0)

# 개별 모델 학습
knn_clf.fit(X_tn, y_tn)
rf_clf.fit(X_tn, y_tn)
dt_clf.fit(X_tn, y_tn)
ada_clf.fit(X_tn, y_tn)

AdaBoostClassifier(n_estimators=100, random_state=0)

### 베이스 러너 정확도 평가

In [14]:
# 기반 모델 예측 세트와 정확도 확인

from sklearn.metrics import accuracy_score

knn_pred = knn_clf.predict(X_te)
rf_pred = rf_clf.predict(X_te)
dt_pred = dt_clf.predict(X_te)
ada_pred = ada_clf.predict(X_te)

print('KNN 정확도 :',accuracy_score(y_te, knn_pred))
print('RF 정확도 :',accuracy_score(y_te, rf_pred))
print('DT 정확도 :',accuracy_score(y_te, dt_pred))
print('ADA부스트 정확도 :',accuracy_score(y_te, ada_pred))

KNN 정확도 : 0.9230769230769231
RF 정확도 : 0.972027972027972
DT 정확도 : 0.8811188811188811
ADA부스트 정확도 : 0.986013986013986


### 메타 모델 빌드: 베이스 러너 결과를 전치하여 각 모델을 컬럼으로 사용

In [15]:
import numpy as np

# 기반 모델의 예측 결과를 스태킹
stacked_pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(stacked_pred.shape)

# transpose를 이용, 행과 열의 위치를 교환, 칼럼 레벨로 각 모델의 예측 결과를 피처로 사용
stacked_pred = np.transpose(stacked_pred)
print(stacked_pred.shape)

(4, 143)
(143, 4)


### 메타 모델로 데이터 예측: 결과 (0.9930)

In [16]:
# 메타 모델은 기반모델의 예측결과를 기반으로 학습

lr_final.fit(stacked_pred, y_te)
final_pred = lr_final.predict(stacked_pred)

print('최종 메타 모델 정확도: ',accuracy_score(y_te, final_pred))

최종 메타 모델 정확도:  0.993006993006993


### 분류 리포트 확인

안해볼 수가 없지. 

In [17]:
from sklearn.metrics import classification_report

prediction_list = [knn_pred, rf_pred, dt_pred, ada_pred]

for i in prediction_list:
    class_report = classification_report(y_te, i)
    print(class_report)
    print("=======================================================")

              precision    recall  f1-score   support

           0       0.88      0.92      0.90        53
           1       0.95      0.92      0.94        90

    accuracy                           0.92       143
   macro avg       0.91      0.92      0.92       143
weighted avg       0.92      0.92      0.92       143

              precision    recall  f1-score   support

           0       0.95      0.98      0.96        53
           1       0.99      0.97      0.98        90

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143

              precision    recall  f1-score   support

           0       0.78      0.94      0.85        53
           1       0.96      0.84      0.90        90

    accuracy                           0.88       143
   macro avg       0.87      0.89      0.88       143
weighted avg       0.90      0.88      0.88       143

              preci

In [18]:
# 최종 모델 분류 리포트

class_report = classification_report(y_te, final_pred)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        53
           1       0.99      1.00      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143



### 결과

스태킹 분류기를 따로 사용하지 않고 바로 이들을 사용하는 방식으로 진행되었다.

knn_pred, rf_pred, dt_pred, ada_pred 로서의 분류 결과도 나쁘지 않았으나, 실제 스태킹 모델의 분류가 수치적으로는 조금 더 정확해 보이기는 한다.

특이한 점은 에이다부스트를 개별 모델로 사용했다는 점인데, 정확도가 디시전 트리보다 좋게 나왔다. 재미있는 부분.

또한 knn과 DT가 똥을 쌌음(?)에도 버스기사가 존재한건지 버스 성능이 좋은건지 메타학습기 정확도가 상당히 높게 나왔다.

---

기존에 익혔던 스태킹 분류기에 각 분류기를 같은 옵션으로 넣었을 때에도 같은 결과가 나오는지 아래에서 해본다.



## Model2: 표준화 진행 없이 StackingClassifier 이용

### 데이터 학습

In [19]:
# 스태킹 모델에 사용할 알고리즘
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# 스태킹을 위한 앙상블 알고리즘
from sklearn.ensemble import StackingClassifier

# 베이스 러너: 개별 ML 모델 객체 생성 
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier(random_state = 0)
ada_clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)

# 메타 모델: 스태킹으로 만들어진 데이터 학습 및 예측 - 로지스틱 리그레션 모형
# 위와 구분짓기 위해 주로 쓰던 네이밍을 사용했다
clf_stacking = StackingClassifier(estimators = [('knn', knn_clf),('rf', rf_clf),('dt', dt_clf),('ada', ada_clf)],
                                  final_estimator = LogisticRegression(C = 10, random_state = 0))

# 데이터 적합
clf_stacking.fit(X_tn, y_tn)

StackingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=4)),
                               ('rf', RandomForestClassifier(random_state=0)),
                               ('dt', DecisionTreeClassifier(random_state=0)),
                               ('ada',
                                AdaBoostClassifier(n_estimators=100,
                                                   random_state=0))],
                   final_estimator=LogisticRegression(C=10, random_state=0))

### 데이터 예측

In [20]:
pred_stacking = clf_stacking.predict(X_te)
print(pred_stacking)

[0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0]


### 정확도 평가: 결과 (0.9720)

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stacking)
print(accuracy)

0.972027972027972


### confusion matrix 확인

In [22]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_stacking)
print(conf_matrix)

[[52  1]
 [ 3 87]]


### 분류 리포트 확인

In [23]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_stacking)
print(class_report)

              precision    recall  f1-score   support

           0       0.95      0.98      0.96        53
           1       0.99      0.97      0.98        90

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



### 결과

정확도가 눈에 띄게(?) 줄었는데 이유를 알고싶다. 

이럴 때에는 코드를 보아야 하는데 본다고 알 수준은 안되니 옵션을 살펴보기로 한다.

```
StackingClassifier(
    estimators,
    final_estimator=None,
    *,
    cv=None,
    stack_method='auto',
    n_jobs=None,
    passthrough=False,
    verbose=0,
)
```

estimators는 그대로 들어갔고, final_estimator도 그대로 들어갔다. 

현재 내가 확인할 수 없는 부분은 cv, stack_method, n_jobs, passthrough, verbose 다섯가지 부분이다.

---



# Mode2: After Standardization

### 트레이닝/테스트 데이터 분할 및 표준화

In [25]:
from sklearn.model_selection import train_test_split              # 데이터 분할을 위한 함수
from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X_data, y_label, random_state = 0) # 데이터 분할

std_scale = StandardScaler()                                      # 표준화 스케일러 선언
std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
X_te_std = std_scale.transform(X_te)

## Model3: 베이스 러너 아웃풋을 전치시켜 메타러너에 입력

### 데이터 학습

In [26]:
# 스태킹 모델에 사용할 알고리즘 호출
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# 베이스 러너: 개별 ML 모델 객체 생성 
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier(random_state = 0)
ada_clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)

# 메타 모델: 스태킹으로 만들어진 데이터 학습 및 예측 - 로지스틱 리그레션 모형
lr_final = LogisticRegression(C = 10, random_state = 0)

# 개별 모델 학습
knn_clf.fit(X_tn_std, y_tn)
rf_clf.fit(X_tn_std, y_tn)
dt_clf.fit(X_tn_std, y_tn)
ada_clf.fit(X_tn_std, y_tn)

AdaBoostClassifier(n_estimators=100, random_state=0)

### 베이스 러너 정확도 평가

In [27]:
# 기반 모델 예측 세트와 정확도 확인

from sklearn.metrics import accuracy_score

knn_pred = knn_clf.predict(X_te_std)
rf_pred = rf_clf.predict(X_te_std)
dt_pred = dt_clf.predict(X_te_std)
ada_pred = ada_clf.predict(X_te_std)

print('KNN 정확도 :',accuracy_score(y_te, knn_pred))
print('RF 정확도 :',accuracy_score(y_te, rf_pred))
print('DT 정확도 :',accuracy_score(y_te, dt_pred))
print('ADA부스트 정확도 :',accuracy_score(y_te, ada_pred))

KNN 정확도 : 0.958041958041958
RF 정확도 : 0.972027972027972
DT 정확도 : 0.8811188811188811
ADA부스트 정확도 : 0.986013986013986


KNN의 정확도가 3%포인트 상승했다.

### 메타 모델 빌드: 베이스 러너 결과를 전치하여 각 모델을 컬럼으로 사용

In [28]:
import numpy as np

# 기반 모델의 예측 결과를 스태킹
stacked_pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(stacked_pred.shape)

# transpose를 이용, 행과 열의 위치를 교환, 칼럼 레벨로 각 모델의 예측 결과를 피처로 사용
stacked_pred = np.transpose(stacked_pred)
print(stacked_pred.shape)

(4, 143)
(143, 4)


### 메타 모델로 데이터 예측: 결과 (0.9930)

In [29]:
# 메타 모델은 기반모델의 예측결과를 기반으로 학습

lr_final.fit(stacked_pred, y_te)
final_pred = lr_final.predict(stacked_pred)

print('최종 메타 모델 정확도: ',accuracy_score(y_te, final_pred))

최종 메타 모델 정확도:  0.993006993006993


어라, 메타모델의 정확도는 그대로 일치한다.

### 분류 리포트 확인

In [30]:
from sklearn.metrics import classification_report

prediction_list = [knn_pred, rf_pred, dt_pred, ada_pred]

for i in prediction_list:
    class_report = classification_report(y_te, i)
    print(class_report)
    print("=======================================================")

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        53
           1       0.97      0.97      0.97        90

    accuracy                           0.96       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.96      0.96       143

              precision    recall  f1-score   support

           0       0.95      0.98      0.96        53
           1       0.99      0.97      0.98        90

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143

              precision    recall  f1-score   support

           0       0.78      0.94      0.85        53
           1       0.96      0.84      0.90        90

    accuracy                           0.88       143
   macro avg       0.87      0.89      0.88       143
weighted avg       0.90      0.88      0.88       143

              preci

In [31]:
# 최종 모델 분류 리포트

class_report = classification_report(y_te, final_pred)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        53
           1       0.99      1.00      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143



### 결과

표준화를 진행하자 KNN의 정확도가 상당수준 상승했다. 하지만 데이터의 절대적 양이 적어서인지는 몰라도 메타 학습기에서의 성능 상승은 볼 수 없었다. 

그렇다면 결과를 전치시켜 그대로 사용한 것 말고, 사이킷런에서 제공하는 Stacking 앙상블 모델을 사용했을 때는 어떨까?

## Model4: 표준화 후 StackingClassifier 이용

### 데이터 학습

In [32]:
# 스태킹 모델에 사용할 알고리즘
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# 스태킹을 위한 앙상블 알고리즘
from sklearn.ensemble import StackingClassifier

# 베이스 러너: 개별 ML 모델 객체 생성 
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier(random_state = 0)
ada_clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)

# 메타 모델: 스태킹으로 만들어진 데이터 학습 및 예측 - 로지스틱 리그레션 모형
# 위와 구분짓기 위해 주로 쓰던 네이밍을 사용했다
clf_stacking = StackingClassifier(estimators = [('knn', knn_clf),('rf', rf_clf),('dt', dt_clf),('ada', ada_clf)],
                                  final_estimator = LogisticRegression(C = 10, random_state = 0))

# 데이터 적합
clf_stacking.fit(X_tn_std, y_tn)

StackingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=4)),
                               ('rf', RandomForestClassifier(random_state=0)),
                               ('dt', DecisionTreeClassifier(random_state=0)),
                               ('ada',
                                AdaBoostClassifier(n_estimators=100,
                                                   random_state=0))],
                   final_estimator=LogisticRegression(C=10, random_state=0))

### 데이터 예측

In [33]:
pred_stacking = clf_stacking.predict(X_te_std)
print(pred_stacking)

[0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0]


### 정확도 평가: 결과 (0.9720)

In [34]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stacking)
print(accuracy)

0.972027972027972


표준화를 시키기 전과 같다.

### confusion matrix 확인

In [35]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_stacking)
print(conf_matrix)

[[51  2]
 [ 2 88]]


### 분류 리포트 확인

In [36]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_stacking)
print(class_report)

              precision    recall  f1-score   support

           0       0.96      0.96      0.96        53
           1       0.98      0.98      0.98        90

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



### 결과

정확도는 같지만 예측에 있어 차이가 있음을 볼 수 있는 부분이 바로 이 대목이다.

컨퓨전 매트릭스가 

```
[[52  1],  [ 3 87]]
```

였지만

```
[[51  2],  [ 2 88]] 
```
이렇게 결과가 변했다. 분명한 차이가 있었음을 알수 있다. 

만약 해당 데이터 포인트를 불러와서 각 모델들이 맞추지 못한 것들이 어떤 데이터인지 뽑아서 볼 수 있으면 더 좋을 것 같다.